In [1]:
import pandas as pd
import sqlite3
import warnings
warnings.simplefilter("ignore")
import requests
from bs4 import BeautifulSoup

In [2]:

movie_bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep='\t', encoding='latin')
reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep='\t', encoding='latin')
movie_tmdb = pd.read_csv('zippedData/tmdb.movies.csv.gz')
movie_budget = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
genres_top = pd.read_csv('top_genres_numbers.csv')
data = pd.read_csv('MovieData.csv')
theatre_weekend = pd.read_csv('weekend_theaters_numbers.csv')


dataset = [movie_bom, movie_info, reviews, movie_tmdb, movie_budget,genres_top, data, theatre_weekend]


In [3]:
conn = sqlite3.connect('zippedData/im.db/im.db')
cursor = conn.cursor()

# Get the main schema
cursor.execute('''
               SELECT name, sql 
               FROM sqlite_master 
               WHERE type='table';
               ''')
schema = cursor.fetchall()

# Print the schema
for name, schema in schema:
    print(f"Table: {name}")
    print(f"Schema: {schema}\n")

Table: movie_basics
Schema: CREATE TABLE "movie_basics" (
"movie_id" TEXT,
  "primary_title" TEXT,
  "original_title" TEXT,
  "start_year" INTEGER,
  "runtime_minutes" REAL,
  "genres" TEXT
)

Table: directors
Schema: CREATE TABLE "directors" (
"movie_id" TEXT,
  "person_id" TEXT
)

Table: known_for
Schema: CREATE TABLE "known_for" (
"person_id" TEXT,
  "movie_id" TEXT
)

Table: movie_akas
Schema: CREATE TABLE "movie_akas" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "title" TEXT,
  "region" TEXT,
  "language" TEXT,
  "types" TEXT,
  "attributes" TEXT,
  "is_original_title" REAL
)

Table: movie_ratings
Schema: CREATE TABLE "movie_ratings" (
"movie_id" TEXT,
  "averagerating" REAL,
  "numvotes" INTEGER
)

Table: persons
Schema: CREATE TABLE "persons" (
"person_id" TEXT,
  "primary_name" TEXT,
  "birth_year" REAL,
  "death_year" REAL,
  "primary_profession" TEXT
)

Table: principals
Schema: CREATE TABLE "principals" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "person_id" TEXT,
  "category" 

In [4]:
for i in dataset:
    display(i.head())
    print()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


,year,rank,genre,movies,gross,tickets,share
0,2023,1,Action,47,"$2,713,081,639","257,652,551",36.43%
1,2023,2,Adventure,24,"$1,455,186,660","138,194,353",19.54%
2,2023,3,Comedy,57,"$1,085,339,861","103,071,184",14.58%
3,2023,4,Drama,124,"$746,957,150","70,936,038",10.03%
4,2023,5,Horror,43,"$710,624,019","67,485,643",9.54%


,movie_name,production_year,movie_odid,production_budget,domestic_box_office,international_box_office,rating,creative_type,source,production_method,genre,sequel,running_time
0,Madea's Family Reunion,2006,8220100,10000000,63257940,62581,PG-13,Contemporary Fiction,Based on Play,Live Action,Comedy,1.0,NaN
1,Krrish,2006,58540100,10000000,1430721,31000000,Not Rated,Science Fiction,Original Screenplay,Live Action,Action,1.0,NaN
2,End of the Spear,2006,34620100,10000000,11748661,175380,PG-13,Historical Fiction,Original Screenplay,Live Action,Drama,0.0,NaN
3,A Prairie Home Companion,2006,24910100,10000000,20342852,6373339,PG-13,Contemporary Fiction,Original Screenplay,Live Action,Comedy,0.0,105.0
4,Saw III,2006,5840100,10000000,80238724,83638091,R,Contemporary Fiction,Original Screenplay,Live Action,Horror,1.0,NaN


,year,rank,movie,distributor,domestic_box_office,opening_weekend_box_office,max_theatre_count
0,2023,1,Barbie,Warner Bros.,"$635,673,355","$162,022,044","4,337"
1,2023,2,The Super Mario Bros. Movie,Universal,"$574,934,330","$146,361,865","4,371"
2,2023,3,Spider-Man: Across the Spider-Verse,Sony Pict…,"$381,311,319","$120,663,589","4,332"
3,2023,4,Guardians of the Galaxy Vol 3,Walt Disney,"$358,995,815","$118,414,021","4,450"
4,2023,5,Oppenheimer,Universal,"$324,052,905","$82,455,420","3,761"


In [5]:
for i in dataset:
    i.info()
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-n

In [6]:
for i in dataset:
    display(i.isna().sum())
    print()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

year       0
rank       0
genre      0
movies     0
gross      0
tickets    0
share      0
dtype: int64

movie_name                    0
production_year               0
movie_odid                    0
production_budget             0
domestic_box_office           0
international_box_office      0
rating                       23
creative_type                13
source                       21
production_method            11
genre                        10
sequel                        2
running_time                114
dtype: int64

year                            0
rank                            2
movie                           2
distributor                   133
domestic_box_office             0
opening_weekend_box_office    700
max_theatre_count              32
dtype: int64

In [7]:
for i in dataset:
    print(round((i.isna().sum().sort_values(ascending = False) / i.shape[0] * 100), 2))
    print()

foreign_gross     39.86
domestic_gross     0.83
studio             0.15
year               0.00
title              0.00
dtype: float64

box_office      78.21
currency        78.21
studio          68.33
writer          28.78
dvd_date        23.01
theater_date    23.01
director        12.76
synopsis         3.97
runtime          1.92
genre            0.51
rating           0.19
id               0.00
dtype: float64

rating        24.83
review        10.22
critic         5.00
publisher      0.57
date           0.00
top_critic     0.00
fresh          0.00
id             0.00
dtype: float64

vote_count           0.0
vote_average         0.0
title                0.0
release_date         0.0
popularity           0.0
original_title       0.0
original_language    0.0
id                   0.0
genre_ids            0.0
Unnamed: 0           0.0
dtype: float64

worldwide_gross      0.0
domestic_gross       0.0
production_budget    0.0
movie                0.0
release_date         0.0
id               

In [8]:
for i in dataset:
    display(i.duplicated().sum())
    print()

0

0

9

0

0

0

0

0

1. We have a dataset of 8 tables:
- movie
- movie_info
- reviews
- movie_tmdb
- movie_budget
- genres_top
- data
- theatre_weekend
2. Our sql database consists of 8 tables:
- writers
- principals
- persons
- movie_ratings
- movie_akas
- directors
- movie_basics

You tube was founded in 2005, Netflix in 200. Streaming platforms changed the industry of cinema. We include movies from 2010

Importing neccessary information from database to dataframe

In [9]:
        
sql_query = pd.read_sql_query ('''
                               SELECT
                               mb.primary_title,
                               mb.start_year,
                               mb.runtime_minutes,
                               mb.genres,
                               mr.averagerating,
                               mr.numvotes,
                               p.primary_name,
                               p.primary_profession
                               FROM movie_akas
                               JOIN movie_ratings mr
                               USING(movie_id)
                               JOIN movie_basics mb
                               USING(movie_id)
                               JOIN writers
                               USING(movie_id)
                               JOIN persons p
                               USING(person_id)
                               WHERE start_year >= 2010
                               AND region = 'US'
                               GROUP BY primary_title
                               ''', conn)

df = pd.DataFrame(sql_query, columns = [
    'primary_title', 'start_year', 'runtime_minutes', 
    'genres', 'averagerating', 'numvotes',
    'primary_name', 'primary_profession']
)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22534 entries, 0 to 22533
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   primary_title       22534 non-null  object 
 1   start_year          22534 non-null  int64  
 2   runtime_minutes     20935 non-null  float64
 3   genres              22470 non-null  object 
 4   averagerating       22534 non-null  float64
 5   numvotes            22534 non-null  int64  
 6   primary_name        22534 non-null  object 
 7   primary_profession  22527 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 1.4+ MB


In [11]:
df.head(10)

,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,primary_name,primary_profession
0,#1 Serial Killer,2013,87.0,Horror,5.6,40,Koji Steven Sakai,"producer,writer,director"
1,#5,2013,68.0,"Biography,Comedy,Fantasy",6.8,6,Ricky Bardy,"producer,director,editor"
2,#Captured,2017,81.0,Thriller,2.8,212,Lizze Gordon,"actress,producer,writer"
3,#DigitalLivesMatter,2016,NaN,Comedy,6.2,19,Cas Sigers-Beedles,"producer,writer,director"
4,#Disneyland60,2015,45.0,Documentary,8.8,5,M.L. Martin,"writer,producer,cinematographer"
5,#Enough,2015,NaN,Drama,2.4,7,Damion Stevenson,"actor,writer,miscellaneous"
6,#FollowFriday,2016,90.0,Thriller,2.8,186,Micho Rutare,"miscellaneous,writer,director"
7,#Horror,2015,101.0,"Crime,Drama,Horror",3.0,3092,Tara Subkoff,"actress,producer,director"
8,#Lucky Number,2015,80.0,Comedy,5.1,425,David Cornue,"producer,writer,composer"
9,#O2LForever,2015,95.0,"Biography,Documentary,Music",9.2,66,Michael Goldfine,"producer,director,writer"


In [12]:
df.runtime_minutes = df.runtime_minutes.fillna(df.runtime_minutes.mean())

In [13]:
df.dropna(subset=['genres', 'primary_profession'], inplace=True)


In [36]:
df.primary_title = df.primary_title.str.lower()
df.genres = df.genres.str.lower()
df.rename(columns={'primary_title': 'movie'})

,movie,start_year,runtime_minutes,genres,averagerating,numvotes,primary_name,primary_profession
0,#1 serial killer,2013,87.000000,horror,5.6,40,Koji Steven Sakai,"producer,writer,director"
1,#5,2013,68.000000,"biography,comedy,fantasy",6.8,6,Ricky Bardy,"producer,director,editor"
2,#captured,2017,81.000000,thriller,2.8,212,Lizze Gordon,"actress,producer,writer"
3,#digitallivesmatter,2016,91.621304,comedy,6.2,19,Cas Sigers-Beedles,"producer,writer,director"
4,#disneyland60,2015,45.000000,documentary,8.8,5,M.L. Martin,"writer,producer,cinematographer"
...,...,...,...,...,...,...,...,...
22529,à 2 heures de paris,2018,80.000000,comedy,5.8,60,Shirley Bousquet,"actress,writer"
22530,ækte vare,2014,89.000000,"drama,music",6.0,479,Anders Ølholm,"writer,director,producer"
22531,évocateur: the morton downey jr. movie,2012,90.000000,documentary,6.9,877,Daniel A. Miller,"producer,writer,director"
22532,última sesión,2010,90.000000,drama,6.2,5,Francesc Paez,"producer,director,actor"


Exploring table genre to define top 10 genres by revenue during 2010-2023

In [39]:
genres_top.genre = genres_top.genre.str.lower()

Converting "gross" and "ticket" to float type

In [15]:
genres_top.gross = genres_top.gross.str.replace(r'[\$,]', '').str.strip().astype(float)
genres_top.tickets = genres_top.tickets.str.replace(r'[\$,]', '').str.strip().astype(float)

Top 10 genres during 2010-2023 period of time

In [16]:
top_10 = genres_top.groupby('genre')['gross'].sum().sort_values(ascending=False).head(10)
total_sum = top_10.sum()
percentage_values = (top_10 / total_sum) * 100
print(round(percentage_values,2))

genre
Action               28.26
Adventure            27.42
Drama                12.90
Comedy               10.49
Thriller/Suspense     8.20
Horror                6.25
Musical               2.32
Romantic Comedy       2.29
Black Comedy          1.09
Documentary           0.77
Name: gross, dtype: float64


Preprocessing table movie_budget

In [17]:
movie_budget.release_date = pd.to_datetime(movie_budget.release_date)

In [40]:
movie_budget.movie = movie_budget.movie.str.lower()

In [18]:
movie_budget = movie_budget[movie_budget.release_date >= '2010-01-01']

In [19]:
movie_budget.production_budget = movie_budget.production_budget.str.replace(r'[\$,]', '').str.strip().astype(float)
movie_budget.domestic_gross= movie_budget.domestic_gross.str.replace(r'[\$,]', '').str.strip().astype(float)

In [20]:
movie_budget = movie_budget.drop(columns=['id','worldwide_gross'])

In [21]:
movie_budget.movie = movie_budget.movie.str.lower()

Preprocessing table theatre_weekend

In [22]:
theatre_weekend = theatre_weekend.drop(columns=[
    'year', 'rank', 'domestic_box_office'])

In [23]:
theatre_weekend.movie = theatre_weekend.movie.str.lower()

Preprocessing table movie_tmdb

In [24]:
movie_tmdb.release_date = pd.to_datetime(movie_tmdb.release_date)

In [25]:
movie_tmdb = movie_tmdb[movie_tmdb.release_date >= '2010-01-01']

In [26]:
movie_tmdb = movie_tmdb.drop(columns=[
    'Unnamed: 0', 'genre_ids', 'id', 'original_language',
    'original_title'
])

In [41]:
movie_tmdb.title = movie_tmdb.title.str.lower()
movie_tmdb.rename(columns={'title':'movie'})

,popularity,release_date,movie,vote_average,vote_count
0,33.533,2010-11-19,harry potter and the deathly hallows: part 1,7.7,10788
1,28.734,2010-03-26,how to train your dragon,7.7,7610
2,28.515,2010-05-07,iron man 2,6.8,12368
4,27.920,2010-07-16,inception,8.3,22186
5,26.691,2010-02-11,percy jackson & the olympians: the lightning t...,6.1,4229
...,...,...,...,...,...
26512,0.600,2018-10-13,laboratory conditions,0.0,1
26513,0.600,2018-05-01,_exhibit_84xxx_,0.0,1
26514,0.600,2018-10-01,the last one,0.0,1
26515,0.600,2018-06-22,trailer made,0.0,1


Preprocessing table data

In [28]:
data = data.drop(columns=[
    'movie_odid', 'production_budget', 'domestic_box_office',
    'international_box_office'
])

In [42]:
data.movie_name = data.movie_name.str.lower()
data.rename(columns={'movie_name':'movie'})

,movie,production_year,rating,creative_type,source,production_method,genre,sequel,running_time
0,madea's family reunion,2006,PG-13,Contemporary Fiction,Based on Play,Live Action,Comedy,1.0,NaN
1,krrish,2006,Not Rated,Science Fiction,Original Screenplay,Live Action,Action,1.0,NaN
2,end of the spear,2006,PG-13,Historical Fiction,Original Screenplay,Live Action,Drama,0.0,NaN
3,a prairie home companion,2006,PG-13,Contemporary Fiction,Original Screenplay,Live Action,Comedy,0.0,105.0
4,saw iii,2006,R,Contemporary Fiction,Original Screenplay,Live Action,Horror,1.0,NaN
...,...,...,...,...,...,...,...,...,...
1931,the nutcracker and the four realms,2018,PG,Fantasy,Based on Folk Tale/Legend/Fairytale,Live Action,Adventure,0.0,99.0
1932,aquaman,2018,PG-13,Super Hero,Based on Comic/Graphic Novel,Live Action,Action,0.0,143.0
1933,ralph breaks the internet,2018,PG,Kids Fiction,Original Screenplay,Digital Animation,Adventure,1.0,112.0
1934,mission: impossible—fallout,2018,PG-13,Contemporary Fiction,Based on TV,Live Action,Action,1.0,147.0


Web parsing data of top 5 streaming platforms

In [30]:
# url = 'https://en.wikipedia.org/wiki/List_of_streaming_media_services'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')
# table = soup.find('table', {'class': 'wikitable'})

In [31]:
# services = []
# descriptions = []

# for row in table.find_all('tr')[1:]:
#     cells = row.find_all('td')
#     if len(cells) > 1:  # Assuming each row has more than one cell
#         service_name = cells[0].get_text(strip=True)
#         description = cells[1].get_text(strip=True)
#         services.append(service_name)
#         descriptions.append(description)

In [32]:
# df = pd.DataFrame({'Service': services, 'Description': descriptions})
# df.head()

Web parsing info about top movie theater chains

In [33]:
# url = 'https://en.wikipedia.org/wiki/List_of_movie_theater_chains'
# response = requests.get(url)

# soup = BeautifulSoup(response.content, 'html.parser')

# table = soup.find('table', {'class': 'wikitable'})

# columns = [th.get_text(strip=True) for th in table.find_all('th')]

In [34]:
# data = []
# for row in table.find_all('tr')[1:]:
#     cells = row.find_all(['td', 'th'])
#     data.append([cell.get_text(strip=True) for cell in cells])

# # Create a DataFrame from the data
# df = pd.DataFrame(data, columns=columns)
# df.head(10)

Creating master table.  
Merging df(imported from database) with:
- movie_budget
- theatre_weekend
- movie_tmbd
- data

In [ ]:
df